<a href="https://colab.research.google.com/github/kaiomudkt/KNN_detector_comodo_via_wifi/blob/main/KNN_dados_redes_wifi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import sklearn
import pandas as pd
import numpy as np

In [ ]:
dados = pd.read_pickle('/content/drive/MyDrive/UFMS/7º Semestre/wifi_dict.pkl')

In [ ]:
df = pd.DataFrame(dados[0])
df

,classe,data_hora,79C7_Oi_Fibra 2.4,CAMILA 2G,#NET-CLARO-WIFI,ADRENALINE - Aldryn,Claudeir 2G,ROSIMARA 2G,Hunorato_2.4G,CRSOTO,SIRIO,Casa,Luiz 2G,5BBA_Fibra_2.4,JOAO 2G,BATCAVERNA,Familia Souza,Maria,9B68_Fibra,GAEL 2G,FamilaSchaffer,Net-Virtua-3944
0,varanda,2021-04-23_09:34:03,67,44,46,16,20,17,22,25,26,0,0,0,0,0,0,0,0,0,0,0
1,varanda,2021-04-23_09:34:06,67,44,46,16,20,17,22,25,26,0,0,0,0,0,0,0,0,0,0,0
2,varanda,2021-04-23_09:34:10,65,45,44,16,22,17,31,25,26,0,0,0,0,0,0,0,0,0,0,0
3,varanda,2021-04-23_09:34:13,65,45,44,16,22,17,31,25,26,0,0,0,0,0,0,0,0,0,0,0
4,varanda,2021-04-23_09:34:17,65,52,57,16,22,19,30,24,26,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,sala,2021-04-23_22:58:23,41,20,20,19,0,0,0,24,43,0,31,26,24,32,31,0,22,0,0,0
896,sala,2021-04-23_22:58:27,40,20,20,19,0,0,0,21,40,0,31,28,25,27,29,0,24,0,0,0
897,sala,2021-04-23_22:58:30,40,20,20,19,0,0,0,21,40,0,31,28,25,27,29,0,24,0,0,0
898,sala,2021-04-23_22:58:35,40,21,19,19,0,0,0,33,39,0,31,28,25,20,32,0,24,0,0,0


In [ ]:
columns_dont_want = ["classe", "data_hora"]
select = [x for x in df.columns if x not in columns_dont_want]
colunas_wifi = df.loc[:, select] # retira as colunas que nao sao de dados dos wifi

In [ ]:
x = df[colunas_wifi.columns].values # valores de cada rede WIFI
y = df['classe'] # comodo: 'classe'

In [ ]:
from sklearn.neighbors import KNeighborsClassifier # I.A. KNN classificadora
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import  accuracy_score,precision_score,recall_score,f1_score,confusion_matrix

In [ ]:
# xtest
# xtest_norm
# #x: amostras de teste
# model.predict_proba(xtest_norm) # Retorna estimativas de probabilidade para os dados de teste X.
# #1. As probabilidades de classe das amostras de entrada. As aulas são ordenadas por ordem lexicográfica.

In [ ]:
# X: "matriz" dados de treinamento
# Y: "matriz" valores alvo

In [ ]:
###### CELULA PARA TESTE E VALICACAO ######
# descobrir o melhor k ?????


# DIVIDI O DATAFRAME (treino e teste )
# n_splits: Número de iterações de rearranjo e divisão.
# test_sizefloat: porcentagem do dataframe usado para teste
# random_state: Controla a aleatoriedade dos índices de treinamento e teste produzidos
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.3, train_size=0.7,random_state=9)
melhores_vizinhos = {}
for qtd_vizinhos in range(2,20):
  # CRIA MODELO, parametro é quantidade de vizinhos
  #p = 2# tipo de distancia
  # peso = # weights
  model = KNeighborsClassifier(n_neighbors=qtd_vizinhos, p=2, weights='uniform') 

  ### NORMALIZAÇÃO ###
  # x.mean(axis=0) # retorna array com a media dos valores de cada coluna wifi
  # x.std(axis=0) # Retorna o array com o desvio padrão da amostra sobre cada coluna solicitado.
  x = x / 70 # onde 70  é a qualidade maxima do sinal, 70 = 100% de qualidade, convertendo para porcentagem, de 0 a 1, como descrito no AVA, quality em porcentagem
  xnorm = (x - x.mean(axis=0)) / x.std(axis=0) # score_Z, normalização

  lacc = []
  # test_index_30: é o index das linhas de 30% do dataset
  # train_index_70: é o idex das linhas de 70% do dataset
  for train_index_70, test_index_30 in sss.split(xnorm,y): # dividi dataframe
      
      ### TESTE E VALIDACAO ###
      x_test_30 = xnorm[test_index_30]
      y_test_30 = y[test_index_30]
      # 30% do dataset de treino divido pela metade, ou seja 15% teste, e 15% validacao
      x_test_15, x_validation_15, y_test_15, y_validation_15 = train_test_split(x_test_30, y_test_30, test_size=0.5, random_state=0)
      model.fit(x_validation_15, y_validation_15) #teste e validacao para descobrir o melhor 'K'

      y_pred = model.predict(x_test_15)
      acc = accuracy_score(y_test_15, y_pred)
      lacc.append(acc)
      media = np.mean(lacc)
      desvio_padrao = np.std(lacc)
      f1_score_para_k = f1_score(y_test_15,y_pred,average='micro')

  melhores_vizinhos[f1_score_para_k] = (qtd_vizinhos, desvio_padrao, media)
  print("qtd_vizinhos: %2d , Média: %4.3f Std: %4.3f " % (qtd_vizinhos,media, desvio_padrao) )

f1_melhor_vizinho = max(melhores_vizinhos)
melhor_vizinho = melhores_vizinhos[f1_melhor_vizinho]
print('melhor quantidade de vizinhos:',melhor_vizinho[0],',media:', melhor_vizinho[2],',desvio padrao:',melhor_vizinho[1])

qtd_vizinhos:  2 , Média: 0.948 Std: 0.010 
qtd_vizinhos:  3 , Média: 0.957 Std: 0.017 
qtd_vizinhos:  4 , Média: 0.941 Std: 0.017 
qtd_vizinhos:  5 , Média: 0.945 Std: 0.014 
qtd_vizinhos:  6 , Média: 0.938 Std: 0.019 
qtd_vizinhos:  7 , Média: 0.942 Std: 0.017 
qtd_vizinhos:  8 , Média: 0.934 Std: 0.028 
qtd_vizinhos:  9 , Média: 0.930 Std: 0.028 
qtd_vizinhos: 10 , Média: 0.911 Std: 0.039 
qtd_vizinhos: 11 , Média: 0.915 Std: 0.038 
qtd_vizinhos: 12 , Média: 0.906 Std: 0.037 
qtd_vizinhos: 13 , Média: 0.908 Std: 0.036 
qtd_vizinhos: 14 , Média: 0.900 Std: 0.036 
qtd_vizinhos: 15 , Média: 0.903 Std: 0.038 
qtd_vizinhos: 16 , Média: 0.891 Std: 0.035 
qtd_vizinhos: 17 , Média: 0.892 Std: 0.040 
qtd_vizinhos: 18 , Média: 0.884 Std: 0.035 
qtd_vizinhos: 19 , Média: 0.887 Std: 0.037 
melhor quantidade de vizinhos: 8 ,media: 0.9340740740740742 ,desvio padrao: 0.027606881709013677


In [ ]:
###### CELULA PARA T R E I N A M E N T O ######

sss = StratifiedShuffleSplit(n_splits=10, test_size=0.3, train_size=0.7,random_state=9)
qtd_vizinhos = 8 # MELHOR K

# CRIA MODELO, parametro é quantidade de vizinhos
model = KNeighborsClassifier(n_neighbors=qtd_vizinhos) 

### NORMALIZAÇÃO ###
x = x / 70 
xnorm = (x - x.mean(axis=0)) / x.std(axis=0)

lacc = []
for train_index_70, test_index_30 in sss.split(xnorm,y):
    x_test_30 = xnorm[test_index_30]
    y_test_30_30 = y[test_index_30]

    ### TREINAMENTO  ###
    xtrain = xnorm[train_index_70]
    ytrain = y[train_index_70]
    model.fit(xtrain,ytrain)

    ###  PRECISAO ###
    y_pred = model.predict(x_test_30)
    acc = accuracy_score(y_test_30, y_pred)
    lacc.append(acc)
    media = np.mean(lacc)
    desvio_padrao = np.std(lacc)

    print("F1_Score: %4.3f :, Precision_Score: %4.3f :, Reall_Score: %4.3f :, Accuracy_score: %4.3f :" % (
        f1_score(y_test_30,y_pred,average='micro'), 
        precision_score(y_test_30,y_pred,average='micro'), 
        recall_score(y_test_30,y_pred,average='micro'), 
        acc) 
    )           
    confusion_matrix(y_test_30,y_pred)                                            
print()
print("qtd_vizinhos: %2d , Média: %4.3f Std: %4.3f " % (qtd_vizinhos,media, desvio_padrao) )
    

F1_Score: 0.326 :, Precision_Score: 0.326 :, Reall_Score: 0.326 :, Accuracy_score: 0.326 :
F1_Score: 0.333 :, Precision_Score: 0.333 :, Reall_Score: 0.333 :, Accuracy_score: 0.333 :
F1_Score: 0.289 :, Precision_Score: 0.289 :, Reall_Score: 0.289 :, Accuracy_score: 0.289 :
F1_Score: 0.337 :, Precision_Score: 0.337 :, Reall_Score: 0.337 :, Accuracy_score: 0.337 :
F1_Score: 0.344 :, Precision_Score: 0.344 :, Reall_Score: 0.344 :, Accuracy_score: 0.344 :
F1_Score: 0.356 :, Precision_Score: 0.356 :, Reall_Score: 0.356 :, Accuracy_score: 0.356 :
F1_Score: 0.337 :, Precision_Score: 0.337 :, Reall_Score: 0.337 :, Accuracy_score: 0.337 :
F1_Score: 0.330 :, Precision_Score: 0.330 :, Reall_Score: 0.330 :, Accuracy_score: 0.330 :
F1_Score: 0.311 :, Precision_Score: 0.311 :, Reall_Score: 0.311 :, Accuracy_score: 0.311 :
F1_Score: 0.967 :, Precision_Score: 0.967 :, Reall_Score: 0.967 :, Accuracy_score: 0.967 :

qtd_vizinhos:  8 , Média: 0.393 Std: 0.192 
